<a href="https://colab.research.google.com/github/Niha002/AI-class/blob/main/Hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup
import numpy as np
import tensorflow as tf
from tensorflow import keras


In [ ]:
!cd drive
!pwd
!ls

/content
drive  sample_data


In [ ]:
# Create a dataset.
dataset = keras.preprocessing.image_dataset_from_directory( '/content/drive/MyDrive/Colab Notebooks', batch_size=64, image_size=(200, 200))
 
# For demonstration, iterate over the batches yielded by the dataset.
for data, labels in dataset:
   print(data.shape)  # (64, 200, 200, 3)
   print(data.dtype)  # float32
   print(labels.shape)  # (64,)
   print(labels.dtype)  # int32


Found 1 files belonging to 1 classes.
(1, 200, 200, 3)
<dtype: 'float32'>
(1,)
<dtype: 'int32'>


In [ ]:
dataset = keras.preprocessing.text_dataset_from_directory('/content/drive/MyDrive/Colab Notebooks', batch_size=64)
 
# For demonstration, iterate over the batches yielded by the dataset.
for data, labels in dataset:
   print(data.shape)  # (64,)
   print(data.dtype)  # string
   print(labels.shape)  # (64,)
   print(labels.dtype)  # int32

Turning strings into sequences of integer word indices

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
 
# Example training data, of dtype `string`.
training_data = np.array([["The 1st AI keras."], [" for prediction."]])
 
# Create a TextVectorization layer instance. It can be configured to either
# return integer token indices, or a dense token representation (e.g. multi-hot
# or TF-IDF). The text standardization and text splitting algorithms are fully
# configurable.
vectorizer = TextVectorization(output_mode="int")
 
# Calling `adapt` on an array or dataset makes the layer generate a vocabulary
# index for the data, which can then be reused when seeing new data.
vectorizer.adapt(training_data)

# After calling adapt, the layer is able to encode any n-gram it has seen before
# in the `adapt()` data. Unknown n-grams are encoded via an "out-of-vocabulary" token.
integer_data = vectorizer(training_data)
print(integer_data)


tf.Tensor(
[[2 7 6 4]
 [5 3 0 0]], shape=(2, 4), dtype=int64)


Turning strings into sequences of one-hot encoded bigrams

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
 
# Example training data, of dtype `string`.
training_data = np.array([["The 1st AI keras."], [" for prediction."]])

# Create a TextVectorization layer instance. It can be configured to either
# return integer token indices, or a dense token representation (e.g. multi-hot
# or TF-IDF). The text standardization and text splitting algorithms are fully
# configurable.
vectorizer = TextVectorization(output_mode="binary", ngrams=2)
 
# Calling `adapt` on an array or dataset makes the layer generate a vocabulary
# index for the data, which can then be reused when seeing new data.
vectorizer.adapt(training_data)
 
# After calling adapt, the layer is able to encode any n-gram it has seen before
# in the `adapt()` data. Unknown n-grams are encoded via an "out-of-vocabulary"
# token.
integer_data = vectorizer(training_data)
print(integer_data)

tf.Tensor(
[[0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0.]], shape=(2, 11), dtype=float32)


Normalizing features

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import Normalization
 
# Example image data, with values in the [0, 500] range
training_data = np.random.randint(0, 500, size=(100, 300, 200, 3)).astype("float32")
 
normalizer = Normalization(axis=-1)
normalizer.adapt(training_data)
 
normalized_data = normalizer(training_data)
print("var: %.4f" % np.var(normalized_data))
print("mean: %.4f" % np.mean(normalized_data))


var: 1.0011
mean: 0.0005


Rescaling & center-cropping images

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import CenterCrop
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
 
# Example image data, with values in the [0, 500] range
training_data = np.random.randint(0, 500, size=(100, 300, 200, 3)).astype("float32")
 
cropper = CenterCrop(height=150, width=150)
scaler = Rescaling(scale=1.0 / 255)
 
output_data = scaler(cropper(training_data))
print("shape:", output_data.shape)
print("min:", np.min(output_data))
print("max:", np.max(output_data))


shape: (100, 150, 150, 3)
min: 0.0
max: 1.9568628


Building Models/Networks with the Keras Functional API

In [ ]:
dense = keras.layers.Dense(units=16)

# Let's say we expect our inputs to be RGB images of arbitrary size
inputs = keras.Input(shape=(200, 200, 3))

from tensorflow.keras import layers
 
# Center-crop images to 150x150
x = CenterCrop(height=150, width=150)(inputs)
# Rescale images to [0, 1]
x = Rescaling(scale=1.0 / 255)(x)
 
# Apply some convolution and pooling layers
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
x = layers.MaxPooling2D(pool_size=(3, 3))(x)
x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x)
 
# Apply global average pooling to get flat feature vectors
x = layers.GlobalAveragePooling2D()(x)
 
# Add a dense classifier on top
num_classes = 10
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

data = np.random.randint(0, 256, size=(64, 200, 200, 3)).astype("float32")
processed_data = model(data)
print(processed_data.shape)

model.summary()


(64, 10)
Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 200, 200, 3)]     0         
                                                                 
 center_crop_3 (CenterCrop)  (None, 150, 150, 3)       0         
                                                                 
 rescaling_4 (Rescaling)     (None, 150, 150, 3)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 49, 49, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 47, 47, 32)        9248      
                                                  

Training Models/Networks with fit()


In [ ]:
# Select Optimizer and Loss Function 
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.CategoricalCrossentropy())
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


In [ ]:
# Train the model/network
model.fit(numpy_array_of_samples, numpy_array_of_labels,
          batch_size=32, epochs=10)

model.fit(dataset_of_samples_and_labels, epochs=10)

Example Networks and Training

In [ ]:
# Get the data as Numpy arrays
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
 
# Build a simple model/network
inputs = keras.Input(shape=(28, 28))
x = layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()
 
# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
 
# Train the model for 1 epoch from Numpy data
batch_size = 64
print("Fit on NumPy data")
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=1)
 
# Train the model for 1 epoch using a dataset
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
print("Fit on Dataset")
history = model.fit(dataset, epochs=1)

11501568/11490434 [==============================] - 0s 0us/step
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 rescaling_2 (Rescaling)     (None, 28, 28)            0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               100480    
                                                                 
 dense_3 (Dense)             (None, 128)               16512     
                                                                 
 dense_4 (Dense)             (None, 10)                1290      
                                                            

In [ ]:
print(history.history)

{'loss': [0.11593519896268845]}


Monitoring metrics

In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)
history = model.fit(dataset, epochs=1)


938/938 [==============================] - 8s 8ms/step - loss: 0.0811 - acc: 0.9753


In [ ]:
# Passing validation data to fit()
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
history = model.fit(dataset, epochs=1, validation_data=val_dataset)


938/938 [==============================] - 8s 8ms/step - loss: 0.0557 - acc: 0.9833 - val_loss: 0.1064 - val_acc: 0.9694


In [ ]:
# Using callbacks for checkpointing (and more)
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='path/to/my/model_{epoch}',
        save_freq='epoch')
]
model.fit(dataset, epochs=2, callbacks=callbacks)


Epoch 1/2
938/938 [==============================] - 8s 9ms/step - loss: 0.0403 - acc: 0.9879
Epoch 2/2
938/938 [==============================] - 5s 5ms/step - loss: 0.0320 - acc: 0.9902


Monitoring training progress with TensorBoard

In [ ]:
callbacks = [
    keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(dataset, epochs=2, callbacks=callbacks)


Epoch 1/2
938/938 [==============================] - 4s 4ms/step - loss: 0.0261 - acc: 0.9921
Epoch 2/2
938/938 [==============================] - 4s 4ms/step - loss: 0.0223 - acc: 0.9932


In [ ]:
# After fit(): evaluating test performance & generating predictions on new data
loss, acc = model.evaluate(val_dataset)  # returns loss and metrics
print("loss: %.2f" % loss)
print("acc: %.2f" % acc)

predictions = model.predict(val_dataset)
print(predictions.shape)

157/157 [==============================] - 1s 8ms/step - loss: 0.1124 - acc: 0.9721
loss: 0.11
acc: 0.97
(10000, 10)


Using fit() with a Custom Training Step

In [ ]:
class CustomModel(keras.Model):
  def train_step(self, data):
    # Unpack the data. Its structure depends on your model and
    # on what you pass to `fit()`.
    x, y = data
    with tf.GradientTape() as tape:
      y_pred = self(x, training=True)  # Forward pass
      # Compute the loss value
      # (the loss function is configured in `compile()`)
      loss = self.compiled_loss(y, y_pred,
                                regularization_losses=self.losses)
    # Compute gradients
    trainable_vars = self.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)
    # Update weights
    self.optimizer.apply_gradients(zip(gradients, trainable_vars))
    # Update metrics (includes the metric that tracks the loss)
    self.compiled_metrics.update_state(y, y_pred)
    # Return a dict mapping metric names to current value
    return {m.name: m.result() for m in self.metrics}
 
# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer='adam', loss='mse', metrics=[...])
 
# Just use `fit` as usual
# model.fit(dataset, epochs=3, callbacks=...)

Debugging your Model

In [ ]:
model.compile(optimizer='adam', loss='mse', run_eagerly=True)

Speeding up Training with Multiple GPUs

In [ ]:
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
 
# Open a strategy scope.
with strategy.scope():
  # Everything that creates variables should be under the strategy scope.
  # In general this is only model construction & `compile()`.
  model = Model(...)
  model.compile(...)
 
# Train the model on all available devices.
train_dataset, val_dataset, test_dataset = get_dataset()
model.fit(train_dataset, epochs=2, validation_data=val_dataset)
 
# Test the model on all available devices.
model.evaluate(test_dataset)

Doing Preprocessing Synchronously 

In [ ]:
# Example training data, of dtype `string`.
samples = np.array([["This is the 1st sample."], ["And here's the 2nd sample."]])
labels = [[0], [1]]
 
# Prepare a TextVectorization layer.
vectorizer = TextVectorization(output_mode="int")
vectorizer.adapt(samples)
 
# Asynchronous preprocessing: the text vectorization is part of the tf.data pipeline.
# First, create a dataset
dataset = tf.data.Dataset.from_tensor_slices((samples, labels)).batch(2)
# Apply text vectorization to the samples
dataset = dataset.map(lambda x, y: (vectorizer(x), y))
# Prefetch with a buffer size of 2 batches
dataset = dataset.prefetch(2)
 
# Our model should expect sequences of integers as inputs
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(input_dim=10, output_dim=32)(inputs)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)
 
model.compile(optimizer="adam", loss="mse", run_eagerly=True)
model.fit(dataset)

# Compare this to doing text vectorization as part of the model:
# Our dataset will yield samples that are strings
dataset = tf.data.Dataset.from_tensor_slices((samples, labels)).batch(2)
 
# Our model should expect strings as inputs
inputs = keras.Input(shape=(1,), dtype="string")
x = vectorizer(inputs)
x = layers.Embedding(input_dim=10, output_dim=32)(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)
 
model.compile(optimizer="adam", loss="mse", run_eagerly=True)
model.fit(dataset)

inputs = keras.Input(shape=(1,), dtype='string')
x = vectorizer(inputs)


1/1 [==============================] - 0s 32ms/step - loss: 0.4951


Finding the Best Model Configuration

In [ ]:
!pip install keras-tuner --upgrade

In [ ]:
def build_model(hp):
    inputs = keras.Input(shape=(784,))
    x = layers.Dense(
        units=hp.Int('units', min_value=32, max_value=512, step=32,
        activation='relu')(inputs)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

import kerastuner
 
tuner = kerastuner.tuners.Hyperband(build_model, objective='val_loss', max_epochs=100,
                                    max_trials=200, executions_per_trial=2, directory='my_dir')

tuner.search(dataset, validation_data=val_dataset)
models = tuner.get_best_models(num_models=2)
tuner.results_summary()